# CCD 관측 자료 전처리 (ysfitsutilpy)


## 데이터 저장

### 데이터 저장 폴더 생성

데이터를 저장할 폴더를 "GSHS_Photometry" 라는 이름으로 생성해 보겠습니다.

* 만약 리눅스 시스템 이라면 shell 명령어로 가능한데, "!"를 붙이면 shell 명령어를 실행할 수 있습니다.
> !mkdir GSHS_Photometry

아래 코드를 실행하면 OS의 영향을 받지 않기 위하여 pathlib을 사용하여 폴더를 생성할 수 있습니다.

In [1]:
import os
from pathlib import Path
BASEPATH = Path("./")
save_dir_name = "GSHS_Photometry"
print(f"BASEPATH: {BASEPATH}")

if not (BASEPATH/save_dir_name).exists():
    os.mkdir(str(BASEPATH/save_dir_name))
    print (f"{str(BASEPATH/save_dir_name)} is created...")
else :
    print (f"{str(BASEPATH/save_dir_name)} is already exist...")

BASEPATH: .
GSHS_Photometry is already exist...


### CCD 관측 데이터 다운로드

경기과학고등학교 천문대에서 CCD로 관측한 이미지 (예제 fits files)을 구글 드라이브에 저장해 놓았습니다. wget 명령을 사용하여 다음과 같이 구글 코랩 [작업 영역]에 다운로드 받아 보겠습니다.

*   bias frame : 5장
*   dark frame : 140sec 5장, 180sec 4장
*   flat frame : 5장
*   object frame : M15 60sec 5장, M35 140sec 10장

파일의 갯수가 많아서 하나의 압축파일로 저장해 놓은 것을 받아 보도록 하겠습니다.



### gdown

구글 드라이브에 저장된 파일을 wget으로 다운받는 경우 용량의 제한이 있습니다. 이를 해결할 수 있는 방법 중의 하나가 gdown을 이용하는 것입니다.

In [3]:
fname = "preprocessing_example.zip"
fid = "1o2YV3Z_G5BIl-MtKg9ng2lTV6W3eHMmU"

# gdown을 이용(나의 구글드라이브에서 공유한 파일을 다운로드)
!gdown {fid} -O {save_dir_name}/{fname}

Downloading...
From (original): https://drive.google.com/uc?id=1o2YV3Z_G5BIl-MtKg9ng2lTV6W3eHMmU
From (redirected): https://drive.google.com/uc?id=1o2YV3Z_G5BIl-MtKg9ng2lTV6W3eHMmU&confirm=t&uuid=f56ad0bb-2156-47ee-b441-f9c7260b028f
To: /home/guitar79/Desktop/KBox/Github/astro_Python/GSHS_Photometry/preprocessing_example.zip
100%|████████████████████████████████████████| 187M/187M [02:51<00:00, 1.09MB/s]


### 데이터 확인

* 코랩을 사용할 경우에는 오른쪽의 [파일]창에서 확인할 수 있습니다.
* linux shell 명령어로 다음과 같이 확인해 볼 수 있습니다.
> ls -l preprocessing_example

OS의 영향을 받지 않고 파이썬으로 확인하는 방법은 아래와 같이 하면 됩니다.

In [4]:
from pathlib import Path
fpaths = sorted(list((BASEPATH/save_dir_name).glob('*.zip')))
print(f"fpaths: {fpaths}")
print(f"len(fpaths): {len(fpaths)}")

fpaths: [PosixPath('GSHS_Photometry/CY-AQR_result_photometry.zip'), PosixPath('GSHS_Photometry/preprocessing_example.zip')]
len(fpaths): 2


### 압축 풀기


In [5]:
import shutil

shutil.unpack_archive(str(fpaths[0]), str(BASEPATH/save_dir_name))
print(str(fpaths[0]), "is unpacked to", str(BASEPATH/save_dir_name))

GSHS_Photometry/CY-AQR_result_photometry.zip is unpacked to GSHS_Photometry


In [6]:
# from zipfile import ZipFile
# with ZipFile(str(fpaths[0]), 'r') as zip_ref:
#     zip_ref.extractall(str(BASEPATH/save_dir_name))
#     print(str(fpath[0]), "is unpacked to", str(BASEPATH/save_dir_name))

### 압축 해제된 파일 확인

shell 명령어로 파일이 생성되었는지 확인해 보자.
역시 마찬가지로 이 창 오른쪽에서 [파일] 목록을 확인해 볼 수 있습니다.

In [7]:
fpaths = sorted(list((BASEPATH/save_dir_name).glob('*.fit*')))
print(f"fpaths: {fpaths}")
print(f"len(fpaths): {len(fpaths)}")

fpaths: [PosixPath('GSHS_Photometry/241GERMANIA_LIGHT_R_2023-11-07-18-38-15_150sec_GSON300_STF-8300M_-19c_1bin.fit'), PosixPath('GSHS_Photometry/46P-WIRTANEN_LIGHT_R_2018-12-13-17-46-15_60sec_FSQ106ED-x73_QSI683ws_-30c_2bin.fit'), PosixPath('GSHS_Photometry/IC2167_LIGHT_L_2023-03-18-13-02-59_600sec_SVX80T-x80_QSI683ws_-20c_1bin.fits'), PosixPath('GSHS_Photometry/IC434_Light_H_2019-10-08-09-38-54_900sec_GSON300_QSI683ws_-15C_1bin.fit'), PosixPath('GSHS_Photometry/M13_Light_L_2021-06-04-13-10-16_090sec_TEC140x75_STL-11000M_-19C_1bin.fit'), PosixPath('GSHS_Photometry/NGC6992_LIGHT_L_2023-06-23-15-37-09_600sec_SVX80T-x80_QSI683ws_-20c_1bin.fits'), PosixPath('GSHS_Photometry/TrES-4b_LIGHT_R_2024-06-13-15-09-52_120sec_GSON300_STF-8300M_-10c_1bin.fit')]
len(fpaths): 7


## fits file display

### zimshow 함수

fits file을 화면에 display 할때 아래의 함수를 사용하면 매우 유용합니다.

In [8]:
from warnings import warn
from astropy.visualization import (
    ImageNormalize,
    LinearStretch,
    ZScaleInterval,
    simple_norm,
)

def znorm(image, stretch=LinearStretch(), **kwargs):
    return ImageNormalize(image, interval=ZScaleInterval(**kwargs), stretch=stretch)

def zimshow(
    ax,
    image,
    stretch=LinearStretch(),
    cmap=None,
    origin="lower",
    zscale_kw={},
    **kwargs
):
    im = ax.imshow(
        image,
        norm=znorm(image, stretch=stretch, **zscale_kw),
        origin=origin,
        cmap=cmap,
        **kwargs
    )
    return im

def norm_imshow(
    ax,
    data,
    origin="lower",
    stretch="linear",
    power=1.0,
    asinh_a=0.1,
    min_cut=None,
    max_cut=None,
    min_percent=None,
    max_percent=None,
    percent=None,
    clip=True,
    log_a=1000,
    invalid=-1.0,
    zscale=False,
    vmin=None,
    vmax=None,
    **kwargs
):
    """Do normalization and do imshow"""
    if vmin is not None and min_cut is not None:
        warn("vmin will override min_cut.")

    if vmax is not None and max_cut is not None:
        warn("vmax will override max_cut.")

    if zscale:
        zs = ImageNormalize(data, interval=ZScaleInterval())
        min_cut = vmin = zs.vmin
        max_cut = vmax = zs.vmax

    if vmin is not None or vmax is not None:
        im = ax.imshow(data, origin=origin, vmin=vmin, vmax=vmax, **kwargs)
    else:
        im = ax.imshow(
            data,
            origin=origin,
            norm=simple_norm(
                data=data,
                stretch=stretch,
                power=power,
                asinh_a=asinh_a,
                min_cut=min_cut,
                max_cut=max_cut,
                min_percent=min_percent,
                max_percent=max_percent,
                percent=percent,
                clip=clip,
                log_a=log_a,
                invalid=invalid
            ),
            **kwargs)
    return im

### display 함수

In [9]:
from astropy.nddata import Cutout2D
import matplotlib.patches as patches

def plot_image_with_cutouts(all_data, cutsizes=101):
    fig_set = plt.figure(figsize=(16, 8))

    def add_subplot(position, data, title):
        ax = plt.subplot2grid((2,4), position, fig=fig_set)
        im = zimshow(ax, data, origin='lower')
        ax.set_ylabel('pixels')
        ax.grid(ls=':')
        ax.set_title(title, fontsize=8)
        plt.colorbar(im, ax=ax, fraction=0.0455, pad=0.04)
        return im

    def add_rectangle(x, y, label):
        rect = patches.Rectangle((x, y), cutsizes,
                                cutsizes, linewidth=1,
                                edgecolor='r', facecolor='none')
        ax1.add_patch(rect)
        ax1.text(x, y, label)

    # 전체 이미지
    ax1 = plt.subplot2grid((2,4), (0,0), colspan=2, rowspan=2, fig=fig_set)
    im1 = zimshow(ax1, all_data, origin='lower', cmap='viridis')
    ax1.set_title('The whole image', fontsize=9)
    plt.colorbar(im1, ax=ax1, fraction=0.0455, pad=0.04)

    # 4개의 부분 이미지
    positions = [(0,2), (0,3), (1,2), (1,3)]
    corners = [(500, all_data.shape[0] - 500),
                (all_data.shape[1] - 500, all_data.shape[0] - 500),
                (500, 500),
                (all_data.shape[1] - 500, 500),
                ]

    for i, (pos, corner) in enumerate(zip(positions, corners), 1):
        cut_hdu = Cutout2D(data=all_data, position=corner, size=(cutsizes, cutsizes))
        add_subplot(pos, cut_hdu.data, f'{i}/4 image area')
        add_rectangle(corner[0], corner[1], f"{i}/4")

    plt.tight_layout()
    plt.show()

# 함수 사용 예
# plot_image_with_cutouts(bias.data)

## ysfitsutilpy

In [10]:
import ysfitsutilpy as yfu

help(yfu)

Help on package ysfitsutilpy:

NAME
    ysfitsutilpy

PACKAGE CONTENTS
    airmass
    astroim
    astrometry
    combutil
    filemgmt
    fitting
    hduutil
    imutil (package)
    misc
    preproc
    tests (package)

SUBMODULES
    docstrings
    imsmooth
    util_comb
    util_lmedian
    util_reject

FUNCTIONS
    warn(...)
        Issue a warning, or maybe ignore it or raise an exception.

        message
          Text of the warning message.
        category
          The Warning category subclass. Defaults to UserWarning.
        stacklevel
          How far up the call stack to make this warning appear. A value of 2 for
          example attributes the warning to the caller of the code calling warn().
        source
          If supplied, the destroyed object which emitted a ResourceWarning
        skip_file_prefixes
          An optional tuple of module filename prefixes indicating frames to skip
          during stacklevel computations for stack frame attribution.

DATA


### make fits file summary

In [11]:
import ysfitsutilpy as yfu

summary = yfu.make_summary(BASEPATH/save_dir_name/"*.fit*",
                           verbose = False,
                           )
if summary is not None :
    print("len(summary):", len(summary))
    print("tyep(summary):", type(summary))
    print("summary:", summary)
    #print(summary["file"][0])

len(summary): 7
tyep(summary): <class 'pandas.core.frame.DataFrame'>
summary:                                                 file  filesize  SIMPLE  \
0  GSHS_Photometry/241GERMANIA_LIGHT_R_2023-11-07...  16980480    True   
1  GSHS_Photometry/46P-WIRTANEN_LIGHT_R_2018-12-1...   4176000    True   
2  GSHS_Photometry/IC2167_LIGHT_L_2023-03-18-13-0...  16706880    True   
3  GSHS_Photometry/IC434_Light_H_2019-10-08-09-38...  16669440    True   
4  GSHS_Photometry/M13_Light_L_2021-06-04-13-10-1...  21430080    True   
5  GSHS_Photometry/NGC6992_LIGHT_L_2023-06-23-15-...  16706880    True   
6  GSHS_Photometry/TrES-4b_LIGHT_R_2024-06-13-15-...  33958080    True   

   BITPIX  NAXIS  NAXIS1  NAXIS2 EXTEND    BZERO     IMAGETYP  ...    FOCTEMP  \
0      16      2    3352    2532   True  32768.0        LIGHT  ...   6.562500   
1      16      2    1663    1252   True  32768.0        LIGHT  ...        NaN   
2      16      2    3326    2504   True  32768.0        LIGHT  ...   5.800000   
3    

### BIAS


In [12]:
bias_fits = summary.loc[summary["IMAGETYP"] == "BIAS"].copy()
bias_fits.reset_index(inplace=True)
bias_fits = bias_fits["file"]
print(type(bias_fits))
print(len(bias_fits))
print(bias_fits)

bias_comb = yfu.group_combine(
                bias_fits.tolist(),
                type_key = ["IMAGETYP"],
                type_val = ["BIAS"],
                group_key = ["EXPTIME"],
                fmt = "master_bias.fits",  # output file name format
                outdir = BASEPATH/save_dir_name,  # output directory (will automatically be made if not exist)
                combine = "med",
                memlimit = 2.e+10,
                verbose = True,
            )

<class 'pandas.core.series.Series'>
0
Series([], Name: file, dtype: object)


IndexError: list index out of range

#### display master bias

In [ ]:
from astropy.io import fits

# 함수 사용 예
hdul = fits.open(BASEPATH/save_dir_name/"master_bias.fits")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

### DARK

In [ ]:
dark_fits = summary.loc[summary["IMAGETYP"] == "DARK"].copy()
dark_fits.reset_index(inplace=True)
dark_fits = dark_fits["file"]
print(type(dark_fits))
print(len(dark_fits))
print(dark_fits)

# Say dark frames have header OBJECT = "calib" && "IMAGE-TYP" = "DARK"
dark_comb = yfu.group_combine(
                dark_fits.tolist(),
                type_key = ["IMAGETYP"],
                type_val = ["DARK"],
                group_key = ["EXPTIME"],
                fmt = "master_dark_{:.0f}sec.fits",  # output file name format
                outdir = BASEPATH/save_dir_name,  # output directory (will automatically be made if not exist)
                combine = "med",
                memlimit = 2.e+10,
                verbose = True,
            )

<class 'pandas.core.series.Series'>
9
0    preprocessing_example/dark-e-2X2-140sec-011.fit
1    preprocessing_example/dark-e-2X2-140sec-012.fit
2    preprocessing_example/dark-e-2X2-140sec-013.fit
3    preprocessing_example/dark-e-2X2-140sec-014.fit
4    preprocessing_example/dark-e-2X2-140sec-015.fit
5    preprocessing_example/dark-e-2X2-180sec-016.fit
6    preprocessing_example/dark-e-2X2-180sec-017.fit
7    preprocessing_example/dark-e-2X2-180sec-018.fit
8    preprocessing_example/dark-e-2X2-180sec-019.fit
Name: file, dtype: object
Group and combine by ['EXPTIME'] (total 2 groups)
* 140.0... (5 files)


	preprocessing_example/master_dark_140sec.fits
* 180.0... (4 files)


	preprocessing_example/master_dark_180sec.fits
..................................(dt = 3.496 s) 2024-09-26T04:50:26.545


#### display master dark

In [ ]:
# 함수 사용 예
hdul = fits.open(BASEPATH/save_dir_name/"master_dark_180sec.fits")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

### FLAT

In [ ]:
flat_fits = summary.loc[summary["IMAGETYP"] == "FLAT"].copy()
flat_fits.reset_index(inplace=True)
flat_fits = flat_fits["file"]
print(type(flat_fits))
print(len(flat_fits))
print(flat_fits)

flat_comb_norm = yfu.group_combine(
                flat_fits.tolist(),
                type_key = ["IMAGETYP"],
                type_val = ["FLAT"],
                group_key = ["FILTER"],
                fmt = "master_flat_{:s}_norm.fits",  # output file name format
                scale="med_sc", #norm
                scale_to_0th=False, #norm
                outdir = BASEPATH/save_dir_name,  # output directory (will automatically be made if not exist)
                combine = "med",
                memlimit = 2.e+10,
                verbose=True,
            )

flat_comb = yfu.group_combine(
                flat_fits.tolist(),
                type_key = ["IMAGETYP"],
                type_val = ["FLAT"],
                group_key = ["FILTER"],
                fmt = "master_flat_{:s}.fits",  # output file name format
                #scale="med_sc", #norm
                #scale_to_0th=False, #norm
                outdir = BASEPATH/save_dir_name,  # output directory (will automatically be made if not exist)
                combine = "med",
                memlimit = 2.e+10,
                verbose=True,
            )

<class 'pandas.core.series.Series'>
5
0    preprocessing_example/flatV-e-2X2-021.fit
1    preprocessing_example/flatV-e-2X2-022.fit
2    preprocessing_example/flatV-e-2X2-023.fit
3    preprocessing_example/flatV-e-2X2-024.fit
4    preprocessing_example/flatV-e-2X2-025.fit
Name: file, dtype: object
Group and combine by ['FILTER'] (total 1 groups)
* V... (5 files)


	preprocessing_example/master_flat_V_norm.fits
..................................(dt = 4.712 s) 2024-09-26T04:50:40.116
Group and combine by ['FILTER'] (total 1 groups)
* V... (5 files)


	preprocessing_example/master_flat_V.fits
..................................(dt = 0.904 s) 2024-09-26T04:50:41.022


#### display master flat

In [ ]:
# 함수 사용 예
hdul = fits.open(BASEPATH/save_dir_name/"master_flat_V.fits")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

#### display master flat norm

In [ ]:
# 함수 사용 예
hdul = fits.open(BASEPATH/save_dir_name/"master_flat_V_norm.fits")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

### Reducing Object Images


In [ ]:
help(yfu.ccdred)

Help on function ccdred in module ysfitsutilpy.preproc:

ccdred(ccd, output=None, extension=None, mbiaspath=None, mdarkpath=None, mflatpath=None, mfrinpath=None, mbias=None, mdark=None, mflat=None, mfrin=None, fringe_flat_fielded=True, fringe_scale=None, fringe_scale_region=None, fringe_scale_kw={}, trimsec=None, gain=1, gain_key='GAIN', gain_unit=Unit("electron / adu"), rdnoise=0, rdnoise_key='RDNOISE', rdnoise_unit=Unit("electron"), exptime_key='EXPTIME', exptime_frin=None, exptime_dark=None, exptime_data=None, dark_scale=False, flat_mask=0, flat_fill=1, flat_norm_value=1, do_crrej=False, crrej_kwargs={'sigclip': 4.5, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'invar': None, 'inbkg': None, 'niter': 4, 'sepmed': False, 'cleantype': 'medmask', 'fs': 'median', 'psffwhm': 2.5, 'psfsize': 7, 'psfbeta': 4.765}, propagate_crmask=False, verbose_crrej=False, verbose_bdf=1, output_verify='fix', overwrite=True, dtype='float32')
    # TODO: add overscan



#### file list

In [ ]:
df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
df_light_m35 = df_light.loc[df_light["OBJECT"] == "m35_677"].copy()
df_light_m35 = df_light_m35.reset_index(drop=True)
print("df_light_m35:", df_light_m35)

df_light_m35:                                      file  filesize  SIMPLE  BITPIX  NAXIS  \
0  preprocessing_example/M35-g3035781.fit   8559360    True      16      2   
1  preprocessing_example/M35-g3035782.fit   8559360    True      16      2   
2  preprocessing_example/M35-g3035783.fit   8559360    True      16      2   
3  preprocessing_example/M35-g3035784.fit   8559360    True      16      2   
4  preprocessing_example/M35-g3035785.fit   8559360    True      16      2   
5  preprocessing_example/M35-g3035786.fit   8559360    True      16      2   
6  preprocessing_example/M35-g3035787.fit   8559360    True      16      2   
7  preprocessing_example/M35-g3035788.fit   8559360    True      16      2   
8  preprocessing_example/M35-g3035789.fit   8559360    True      16      2   
9  preprocessing_example/M35-g3035790.fit   8559360    True      16      2   

   NAXIS1  NAXIS2  BSCALE    BZERO OBSERVAT  ... TELESCOP XBINNING YBINNING  \
0    2048    2048     1.0  32768.0     GSHS  ...

#### No dark scale

In [ ]:
for _, row in df_light_m35.iterrows():

    fpath = Path(row["file"])
    ccd = yfu.load_ccd(fpath)
    filt = ccd.header["FILTER"]
    expt = ccd.header["EXPTIME"]
    red = yfu.ccdred(
        ccd,
        output = BASEPATH /save_dir_name/ f"reduce_{fpath.name}",
        mbiaspath = BASEPATH / save_dir_name / "master_bias.fits",
        mdarkpath = BASEPATH / save_dir_name / f"master_dark_{expt:.0f}sec.fits",
        mflatpath = BASEPATH / save_dir_name / f"master_flat_{filt.upper()}_norm.fits",
        dark_scale = False,  # default False
        # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
        overwrite=True
        )

HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.013 s) 2024-09-26T04:50:50.069
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_140sec.fits)
HISTORY  ..................................(dt = 0.014 s) 2024-09-26T04:50:50.088
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:50:50.103
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.021 s) 2024-09-26T04:50:50.114
Writing FITS to preprocessing_example/reduce_M35-g3035781.fit... Saved.
HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.055 s) 2024-09-26T04:50:50.612
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM 

#### Dark scale

In [ ]:
for _, row in df_light_m35.iterrows():

    fpath = Path(row["file"])
    ccd = yfu.load_ccd(fpath)
    filt = ccd.header["FILTER"]
    expt = ccd.header["EXPTIME"]

    red = yfu.ccdred(
        ccd,
        output = BASEPATH / save_dir_name/ f"reduce_darksc_{fpath.name}",
        mbiaspath = BASEPATH /save_dir_name / "master_bias.fits",
        mdarkpath = BASEPATH / save_dir_name / f"master_dark_180sec.fits",
        mflatpath = BASEPATH / save_dir_name / f"master_flat_{filt.upper()}_norm.fits",
        dark_scale = True,
        exptime_dark = 180,
        # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
        overwrite=True,
        )

HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.011 s) 2024-09-26T04:50:52.647
HISTORY  [yfu.darkcor] Dark scaled by exptime: (t_data/t_dark) = (140.000/180.000) = 0.778
HISTORY  .................................................2024-09-26T04:50:52.660
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_180sec.fits)
HISTORY  ..................................(dt = 0.016 s) 2024-09-26T04:50:52.666
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:50:52.676
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.015 s) 2024-09-26T04:50:52.684
Writing FITS to preprocessing_example/reduce_darksc_M35-g3035781.fit... Saved.
HISTORY  [yfu.biascor] Bias subtracted (

## 확인

전처리한 결과를 비교해 보겠습니다.


### 전처리 전

In [ ]:
# 함수 사용 예
from astropy.io import fits
fpath = Path(df_light_m35['file'][0])
hdul = fits.open(fpath)

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

### 전처리 (No dark scale)

In [ ]:
# 함수 사용 예
from astropy.io import fits
fpath = Path(df_light_m35['file'][0])
hdul = fits.open(fpath.parent/f"reduce_{fpath.name}")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

### 전처리 (Dark scale)

In [ ]:
# 함수 사용 예
fpath = Path(df_light_m35['file'][0])
hdul = fits.open(fpath.parent/f"reduce_darksc_{fpath.name}")

plot_image_with_cutouts(hdul[0].data)

Output hidden; open in https://colab.research.google.com to view.

##(과제)

1. M15를 관측한 5개의 파일 모두를 전처리를 수행하시오. (dark scale을 적용해야 함)

In [ ]:
# (과제) 아래에 코딩을 완성하여 제출하시오.

df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
df_light_m15 = df_light.loc[df_light["OBJECT"] == "M15"].copy()
df_light_m15 = df_light_m15.reset_index(drop=True)
print("df_light_m15:", df_light_m15)

# (과제) 아래에 코딩을 완성하여 제출하시오.


DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                 


df_light_m15:                                    file  filesize  SIMPLE  BITPIX  NAXIS  \
0  preprocessing_example/M15-g00001.fit   8395200    True      16      2   
1  preprocessing_example/M15-g00002.fit   8395200    True      16      2   
2  preprocessing_example/M15-g00003.fit   8395200    True      16      2   
3  preprocessing_example/M15-g00004.fit   8395200    True      16      2   
4  preprocessing_example/M15-g00005.fit   8395200    True      16      2   

   NAXIS1  NAXIS2  BSCALE    BZERO OBSERVAT  ... TELESCOP XBINNING YBINNING  \
0    2048    2048     1.0  32768.0     GSHS  ...        -      2.0      2.0   
1    2048    2048     1.0  32768.0     GSHS  ...        -      2.0      2.0   
2    2048    2048     1.0  32768.0     GSHS  ...        -      2.0      2.0   
3    2048    2048     1.0  32768.0     GSHS  ...        -      2.0      2.0   
4    2048    2048     1.0  32768.0     GSHS  ...        -      2.0      2.0   

  EXPOSURE TELESCOPE APATURE FLIPSTAT  XPIXSZ  YPIXSZ 

HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.013 s) 2024-09-26T04:51:07.513
HISTORY  [yfu.darkcor] Dark scaled by exptime: (t_data/t_dark) = (60.000/180.000) = 0.333
HISTORY  .................................................2024-09-26T04:51:07.529
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_180sec.fits)
HISTORY  ..................................(dt = 0.018 s) 2024-09-26T04:51:07.536
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:51:07.550
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.021 s) 2024-09-26T04:51:07.561
Writing FITS to preprocessing_example/reduce_darksc_M15-g00001.fit... Saved.


HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.013 s) 2024-09-26T04:51:07.751
HISTORY  [yfu.darkcor] Dark scaled by exptime: (t_data/t_dark) = (60.000/180.000) = 0.333
HISTORY  .................................................2024-09-26T04:51:07.766
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_180sec.fits)
HISTORY  ..................................(dt = 0.017 s) 2024-09-26T04:51:07.773
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:51:07.787
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.021 s) 2024-09-26T04:51:07.798
Writing FITS to preprocessing_example/reduce_darksc_M15-g00002.fit... Saved.


HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.012 s) 2024-09-26T04:51:08.010
HISTORY  [yfu.darkcor] Dark scaled by exptime: (t_data/t_dark) = (60.000/180.000) = 0.333
HISTORY  .................................................2024-09-26T04:51:08.030
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_180sec.fits)
HISTORY  ..................................(dt = 0.020 s) 2024-09-26T04:51:08.037
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:51:08.051
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.025 s) 2024-09-26T04:51:08.065
Writing FITS to preprocessing_example/reduce_darksc_M15-g00003.fit... Saved.


HISTORY  [yfu.biascor] Bias subtracted (BIASFRM = preprocessing_example/master_bias.fits)
HISTORY  ..................................(dt = 0.014 s) 2024-09-26T04:51:08.276
HISTORY  [yfu.darkcor] Dark scaled by exptime: (t_data/t_dark) = (60.000/180.000) = 0.333
HISTORY  .................................................2024-09-26T04:51:08.292
HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = preprocessing_example/master_dark_180sec.fits)
HISTORY  ..................................(dt = 0.019 s) 2024-09-26T04:51:08.300
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-26T04:51:08.314
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = preprocessing_example/master_flat_V_norm.fits)
HISTORY  ..................................(dt = 0.022 s) 2024-09-26T04:51:08.326
Writing FITS to preprocessing_example/reduce_darksc_M15-g00004.fit... Saved.
HISTORY  [yfu.biascor] Bias subtracted (BIA

##(과제)

2. 그리고 전처리된 M15 5개의 파일을 모두 display 하시오.

In [ ]:
# (과제) 아래에 코딩을 완성하여 제출하시오.

from astropy.io import fits

df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
df_light_m15 = df_light.loc[df_light["OBJECT"] == "M15"].copy()
df_light_m15 = df_light_m15.reset_index(drop=True)

for _, row in df_light_m15.iterrows():
    fpath = Path(row["file"])
    hdul = fits.open(fpath.parent/f"reduce_darksc_{fpath.name}")
    plot_image_with_cutouts(hdul[0].data)



Output hidden; open in https://colab.research.google.com to view.